In [2]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import tensorflow_text as text
if tf.test.gpu_device_name():
      print(f'Default GPU Device:{tf.test.gpu_device_name()}')  
else:
      print("Please check your installation")

Please check your installation


In [3]:
df = pd.read_csv("spam_or_not_spam.csv")
df.head(6)

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0
5,i just had to jump in here as carbonara is on...,0


In [5]:
df.groupby('label').describe()

email                                                               
      count unique                                                top freq
label                                                                     
0      2500   2445                     url URL date not supplied URL    10
1       499    427  lowest rates available for term life insurance...    5

In [7]:
df_spam = df[df['label']==1]
df_spam.shape

(500, 2)

In [8]:
df_ham = df[df['label']==0]
df_ham.shape

(2500, 2)

In [9]:
df_ds = df_ham.sample(df_spam.shape[0])
df_ds.shape

(500, 2)

In [10]:
df_new = pd.concat([df_ds,df_spam])
df_new.shape

(1000, 2)

In [11]:
df_new['label'].value_counts()

label
0    500
1    500
Name: count, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_new['email'], df_new['label'], stratify=df_new['label'])

In [15]:
x_train.head(6)
prep_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

In [16]:
bert_pp = hub.KerasLayer(prep_url)
bert_enc = hub.KerasLayer(encoder_url)

In [17]:
def get_sen(sentence):
    pp_text = bert_pp(sentence)
    return bert_enc(pp_text)['pooled_output']
e = get_sen(["greece", "profit", "persia","economics"])

In [18]:
e

<tf.Tensor: shape=(4, 768), dtype=float32, numpy=
array([[-0.93499964, -0.49616048, -0.562756  , ..., -0.28465608,
        -0.67491376,  0.9201721 ],
       [-0.906435  , -0.36100864, -0.21344265, ..., -0.03724111,
        -0.65825623,  0.8938946 ],
       [-0.65437466, -0.14296155,  0.74142486, ...,  0.5443639 ,
        -0.48317453,  0.62695843],
       [-0.94314086, -0.6446065 , -0.9248845 , ..., -0.8582408 ,
        -0.72283983,  0.8971318 ]], dtype=float32)>

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity([e[0]],[e[1]])

array([[0.9581166]], dtype=float32)

In [26]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
pp_text = bert_pp(text_input)
output = bert_enc(pp_text)

l = tf.keras.layers.Dropout(0.1, name = 'dropout')(output['pooled_output'])
l1 = tf.keras.layers.Dense(1, activation='sigmoid', name = 'output')(l)

mod = tf.keras.Model(inputs=[text_input], outputs=[l1])

mod.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [30]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
    tf.keras.metrics.Precision(name = 'precision'),
    tf.keras.metrics.Recall(name = 'recall')
]
mod.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics=METRICS)


In [32]:
mod.fit(x_train, y_train, epochs=3)

Epoch 1/3


UnknownError: Graph execution error:

KeyError: '[51, 117, 237, 552, 648, 370, 256, 639, 151, 620, 457, 720, 466, 630, 43, 297, 488, 169, 724, 224, 386, 67, 405, 397] not in index'
Traceback (most recent call last):

  File "c:\Users\DELL\anaconda3\envs\cudaproj\lib\site-packages\tensorflow\python\ops\script_ops.py", line 269, in __call__
    return func(device, token, args)

  File "c:\Users\DELL\anaconda3\envs\cudaproj\lib\site-packages\tensorflow\python\ops\script_ops.py", line 147, in __call__
    outputs = self._call(device, args)

  File "c:\Users\DELL\anaconda3\envs\cudaproj\lib\site-packages\tensorflow\python\ops\script_ops.py", line 154, in _call
    ret = self._func(*args)

  File "c:\Users\DELL\anaconda3\envs\cudaproj\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "c:\Users\DELL\anaconda3\envs\cudaproj\lib\site-packages\keras\engine\data_adapter.py", line 513, in py_method
    return [slice_array(inp)